## 3.6 손글씨 숫자 인식

### 3.6.1 MNIST 데이터셋

#### MNIST
 - 손글씨 숫자 이미지 집합
 - 실험용 데이터로 많이 사용
 - 0~9까지의 숫자 이미지로 구성(훈련 60,000장, 시험 10,000장)
 - 28 x 28 크기의 회색조 이미지(1채널)로, 각 픽셀은 0에서 255까지의 값을 취한다.

#### 모듈
- sys 모듈을 이용해 부모디렉터리의 파일을 가져올 수 있도록 설정
(sys.path.append(os.pardir))
- dataset 디렉토리에 있는 mnist.py의 load_mnist 함수를 import 한다.
    - 첫번째는 인터넷에서 다운로드하며 이후부터는 pickle 파일에 의해 로컬에 저장된 파일을 읽어온다.
- **normalize** : 입력 이미지의 픽셀값을 0.0 ~ 1.0 사이의 값으로 정규화 여부를 정함.
(정규화하면 overflow를 방지할 수 있고, 학습 속도를 향상시킬 수 있다.)
- **flatten** : 입력 이미지를 평탄값 즉, 1차원 배열로 생성 여부를 정함.
- **fromarray** : 넘파이로 저장된 이미지 데이터를 PIL용 데이터 객체로 변환.

In [1]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

In [2]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [3]:
(x_train, t_train), (x_test, t_test) = \
    load_mnist(flatten=True, normalize=False)

In [4]:
img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


- 이미지를 1차원 넘파이 배열로 저장했기 때문에 이미지를 다시 표시하기 위해선 28x28 크기의 배열로 다시 변형해야 한다.

### 3.6.2 신경망의 추론 처리

- 입력층 뉴런 784개(28x28), 출력층 뉴런 10개(0~9까지)로 구성
- 은닉층은 총 두 개
- 첫번째 은닉층은 50개의 뉴런
- init_network()에서는 pickle 파일인 sample_weight.pkl에 저장된 *학습된 가중치, 매개변수*를 읽는다. 가중치와 편향 매개변수가 딕셔너리 변수로 저장되어 있다.
- 이미 학습된 매개변수를 이용하기 때문에 test_set만 불러와서 진행한다.

In [5]:
import pickle

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True)
    # normalize=True 시 overflow 방지 + 학습 속도 향상 (크기가 작아지기 때문.)
    return x_test, t_test


def init_network():
    with open("./ch03/sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
    return network


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # 오버플로 방지
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    return y

**평가**   
   
신경망에 의한 추론을 수행해보고, **정확도**(분류가 얼마나 올바른가)를 평가해보자.   
   
- 가장 먼저 mnist 데이터 셋을 얻고 네트워크를 생성
- for문을 돌려 x에 저장된 이미지 데이터를 1장씩 꺼내 predict()함수로 분류
- predict() 함수는 각 레이블의 확률을 넘파이 배열로 반환
- 예를 들어 [0.1, 0.3, ...] 이라 하면 이미지가 숫자 0일 확률은 10%라고 해석
- 그런 다음 np.argmax() 함수를 이용해 배열에서 값이 가장 큰 원소의 인덱스를 구한다.

In [6]:
x, t = get_data() # ((x_train, t_train), (x_test, t_test))
network = init_network() # 가중치 W, 바이어스 b

error_index = []
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다.
    if p == t[i]:
        accuracy_cnt += 1
    if p != t[i]:
        error_index.append(i)

print("Accuracy:" + str(float(accuracy_cnt) / len(x)) + "\n")
print("Error index:\n" + str(error_index))

Accuracy:0.9352

Error index:
[8, 33, 66, 92, 124, 149, 217, 233, 241, 245, 247, 259, 290, 300, 313, 320, 321, 340, 341, 352, 358, 362, 381, 445, 448, 449, 478, 479, 495, 502, 507, 511, 531, 543, 551, 565, 569, 578, 582, 591, 610, 619, 627, 629, 659, 684, 691, 707, 717, 720, 740, 791, 810, 839, 844, 857, 881, 882, 898, 924, 938, 939, 947, 950, 956, 965, 982, 999, 1014, 1032, 1039, 1044, 1050, 1062, 1068, 1082, 1107, 1112, 1114, 1119, 1124, 1181, 1191, 1192, 1194, 1198, 1204, 1206, 1224, 1226, 1232, 1242, 1247, 1251, 1256, 1260, 1283, 1289, 1299, 1319, 1325, 1326, 1328, 1337, 1364, 1378, 1393, 1410, 1413, 1429, 1433, 1440, 1444, 1465, 1466, 1467, 1494, 1500, 1522, 1525, 1527, 1530, 1549, 1553, 1559, 1569, 1581, 1601, 1609, 1621, 1634, 1640, 1671, 1678, 1681, 1696, 1709, 1717, 1722, 1732, 1737, 1751, 1754, 1759, 1765, 1772, 1773, 1782, 1790, 1800, 1813, 1828, 1850, 1857, 1865, 1878, 1880, 1901, 1913, 1917, 1930, 1938, 1952, 1955, 1956, 1970, 1973, 1981, 1984, 2016, 2024, 2035, 2040, 2043

*__정규화__*   
데이터를 특정 범위로 변환하는 처리. 예를 들어 0 - 255 범위인 각 픽셀의 값을 0.0 - 1.0의 범위로 변환한다.

*__전처리__*   
신경망의 입력 데이터에 특정 변환을 가하는 것. 입력 이미지 데이터에 대한 전처리 작업으로 정규화를 수행한 것

### 3.6.3 배치 처리
**Weight와 bias 값을 최대 효율로 학습**시키기 위해 사용한다.   
batch size : 1 step에서 사용하는 데이터 수   
step : Weight, bias를 1회 업데이트 하는 것   
epoch : 전체 데이터를 1바퀴 돌며 학습

In [7]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

In [8]:
x.shape

(10000, 784)

In [9]:
x[0].shape

(784,)

In [10]:
W1.shape

(784, 50)

In [11]:
W2.shape

(50, 100)

In [12]:
W3.shape

(100, 10)

In [13]:
'''
# 신경망 각 층의 배열 형상의 추이
 X     W1      W2      W3      ->    Y   
784  784x50  50x100  100x10         10 
'''

'\n# 신경망 각 층의 배열 형상의 추이\n X     W1      W2      W3      ->    Y   \n784  784x50  50x100  100x10         10 \n'

In [14]:
'''
# 배치 처리를 위한 배열들의 형상 추이
   X       W1      W2      W3      ->     Y
100x784  784x50  50x100  100x10        100x10
'''

'\n# 배치 처리를 위한 배열들의 형상 추이\n   X       W1      W2      W3      ->     Y\n100x784  784x50  50x100  100x10        100x10\n'

**배치 처리 구현**   
- 이미지를 100장씩 끊어서 처리한다.
- 이전의 방식은 한 장씩 끊어서 처리하였음.
- range() 함수가 반환하는 리스트를 바탕으로 x[i:i+batch_size]에서 데이터를 묶는다.
- x[0:100], x[100:200], ... 와 같이 앞에서부터 100장씩 묶는다.
- argmax() 최댓값의 인덱스를 가져온다.최댓값의 인덱스의 기준을 명시하기 위해 axis=1 변수를 지정해준다.

In [15]:
x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


*__argmax__*   
axis의 값은 기준값을 의미한다. 0일 때는 row 방향으로, 1일 때는 column 방향으로, 2일 때는 z축을 기준으로 함.

In [16]:
x = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6], [0.2, 0.5, 0.3], [0.8, 0.1, 0.1], [0.9, 0.9, 0.9]])
np.argmax(x, axis=0)

array([4, 4, 4], dtype=int64)

In [17]:
np.argmax(x, axis=1)

array([1, 2, 1, 0, 0], dtype=int64)

#### 정리
    - 각 층의 뉴런들이 다음 층의 뉴런으로 신호를 전달하는 점에서 앞 장의 퍼셉트론과 같다.
    - 다음 뉴런으로 갈 때 신호를 변화시키는 활성화 함수에 큰 차이가 존재한다.
    - 신경망에서는 매끄럽게 변화하는 시그모이드 함수를 활성화 함수로 사용한다.
    - 퍼셉트론에서는 갑자기 변화하는 계단 함수를 활성화 함수로 사용한다.